In [1]:
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

os.environ['CLMM_MODELING_BACKEND'] = 'nc'

__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import timedelta
from astropy import units
from numpy import random
plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology
from clmm.support import mock_data as mock
from clmm.utils import convert_units
from clmm.support.sampler import samplers
from scipy.stats import kstest
import scipy.integrate as integrate

In [2]:
np.random.seed(11)

cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)

cluster_m     = 1.e15
logm = np.log(cluster_m)/np.log(10)
cluster_z     = 0.4
concentration = 4
ngals         = 10000
Delta         = 200
cluster_ra    = 0.0
cluster_dec   = 0.0

shapenoise = 1e-3

gal_cat = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 'chang13', zsrc_min = cluster_z + 0.1, shapenoise=shapenoise, photoz_sigma_unscaled=0.05, ngals=ngals, cluster_ra=cluster_ra, cluster_dec=cluster_dec)

gc = clmm.GalaxyCluster("CL_noisy_z", cluster_ra, cluster_dec, cluster_z, gal_cat)

In [3]:
gc.compute_tangential_and_cross_components(geometry="flat")
radius = convert_units(gc.galcat['theta'], 'radians', 'Mpc', redshift=gc.z, cosmo=cosmo)

In [4]:
def shear_log_likelihood(logm, gc, radius, shapenoise, cosmo, sigma=1, concentration=4, delta=200, model='nfw'):

    m = float(10.**logm)
    gt_th = clmm.compute_reduced_tangential_shear(radius, m, concentration, gc.z, gc.galcat['z'], cosmo, delta_mdef=200, halo_profile_model='nfw')
    
    gt_th_kde = Ncm.StatsDist1dEPDF.new_full(2000, Ncm.StatsDist1dEPDFBw.ROT, 1, 0.001)

    for gt in gt_th:
        if gt != 0:
            gt_th_kde.add_obs(gt)
            
    gt_th_kde.prepare()

    gt_obs = gc.galcat['et']

    log_likelihood = 0

    for gt in gt_obs:
        integrand = lambda gt_th : gt_th_kde.eval_p(gt_th)*np.exp(-(gt_th - gt)**2/(shapenoise))/np.sqrt(2*np.pi)/shapenoise
        gt_likelihood = integrate.quad(integrand, -np.inf, np.inf)
        log_likelihood += np.log(gt_th_kde.eval_p(gt))
    
    return -2*log_likelihood


In [5]:
def shear_log_likelihood(logm, gc, radius, shapenoise, cosmo, concentration=4, delta=200, model='nfw'):

    m = float(10.**logm)
    N = len(gc.galcat)
    gt_th = clmm.compute_reduced_tangential_shear(radius, m, concentration, gc.z, gc.galcat['z'], cosmo, delta_mdef=200, halo_profile_model='nfw')
    gt_sigma = np.std(gt_th)
    iqr = np.subtract(*np.percentile(gt_th, [75, 25]))
    h = 0.9*min(gt_sigma, iqr/1.34)*N**(-1/5)

    gt_obs = gc.galcat['et']

    log_likelihood = 0

    for et in gt_obs:
        et_likelihood = 0
        for gt in gt_th:
            et_likelihood += np.exp(-(gt**2/(2*h**2) + et**2/(2*shapenoise**2) - (gt/(2*h**2) + et/(2*shapenoise**2))**2*(2*h**2*shapenoise**2)/(h**2+shapenoise**2)))
        likelihood += np.log(et_likelihood)
    
    
    return log_likelihood


In [6]:
logm_0 = random.uniform(13., 17., 1)[0]

logm_est = samplers['basinhopping'](shear_log_likelihood, logm_0, minimizer_kwargs={'args':(gc, radius, shapenoise, cosmo)})

/tmp/ipykernel_41111/1573609088.py:19: RuntimeWarning: overflow encountered in double_scalars
  likelihood *= et_likelihood
/home/caiooliveira/.local/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/tmp/ipykernel_41111/1573609088.py:19: RuntimeWarning: invalid value encountered in double_scalars
  likelihood *= et_likelihood


KeyboardInterrupt: 

In [ ]:
print(10.**logm_est)

[1.18165893e+15]


In [ ]:
def shear_log_likelihood(logm, gc, radius, shapenoise, cosmo, concentration=4, delta=200, model='nfw'):

    m = float(10.**logm)
    N = gc.ngals
    gt_th = clmm.compute_reduced_tangential_shear(radius, m, concentration, gc.z, gc.galcat['z'], cosmo, delta_mdef=200, halo_profile_model='nfw')
    gt_sigma = np.std(gt_th)
    h = 0.9*gt_sigma*np.pow(N, -1/5)

    gt_obs = gc.galcat['et']

    likelihood = 1

    for et in gt_obs:
        et_likelihood = 0
        for gt in gt_th:
            et_likelihood += np.exp(-(gt**2/(2*h**2) + et**2/(2*shapenoise**2) - (gt/(2*h**2) + et/(2*shapenoise**2))**2*(2*h**2*shapenoise**2)/(h**2+shapenoise**2)))
        et_likelihood/(N*np.sqrt(2*np.pi*(h**2+shapenoise**2)))
        likelihood*et_likelihood
    
    
    return -2*np.log(likelihood)
